In [1]:
import pickle
import os
from pathlib import Path

from matchms.importing import load_from_mgf
from tensorflow import keras
import pandas as pd

from ms2deepscore import SpectrumBinner
from ms2deepscore.data_generators import DataGeneratorAllSpectrums
from ms2deepscore.models import SiameseModel
from ms2deepscore import MS2DeepScore
root_folder = "C:/Users/jonge094/PycharmProjects/PhD_MS2Query/ms2query/data/gnps_24_11_2021/positive_mode"



## load in data

In [2]:
outfile = os.path.join(root_folder, "GNPS_24_11_2021_pos_train.pickle")
with open(outfile, 'rb') as file:
    training_spectra = pickle.load(file)


outfile = os.path.join(root_folder, "GNPS_24_11_2021_pos_tanimoto_scores.pickle")
with open(outfile, 'rb') as file:
    tanimoto_scores = pickle.load(file)


Add validation data together

In [3]:

outfile = os.path.join(root_folder, "GNPS_24_11_2021_pos_val_250_inchikeys.pickle")
with open(outfile, 'rb') as file:
    validation_spectra_250 = pickle.load(file)
    
outfile = os.path.join(root_folder, "GNPS_24_11_2021_pos_val_3000_spectra.pickle")
with open(outfile, 'rb') as file:
    validation_spectra_3000 = pickle.load(file)

validation_spectra = validation_spectra_250 + validation_spectra_3000

### Create binned spectra

In [4]:
spectrum_binner = SpectrumBinner(10000, mz_min=10.0, mz_max=1000.0, peak_scaling=0.5, allowed_missing_percentage =10)
binned_train_spectrums = spectrum_binner.fit_transform(training_spectra)
binned_val_spectrums = spectrum_binner.transform(validation_spectra)

Collect spectrum peaks...
Calculated embedding dimension: 9948.
Convert spectrums to binned spectrums...


Create BinnedSpectrum instances: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 6334/6334 [00:00<00:00, 47421.86it/s]


### Create data generators

In [5]:
dimension = len(spectrum_binner.known_bins)
train_data_generator = DataGeneratorAllSpectrums(binned_train_spectrums, tanimoto_scores,
                                           dim=dimension)
val_data_generator = DataGeneratorAllSpectrums(binned_val_spectrums, tanimoto_scores,
                                           dim=dimension)

19917 out of 20417 InChIKeys found in selected spectrums.
3250 out of 20417 InChIKeys found in selected spectrums.


In [6]:
model = SiameseModel(spectrum_binner, base_dims=(500, 500), embedding_dim=200,
                     dropout_rate=0.2)
model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.001))
model.summary()

Model: "base"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
base_input (InputLayer)      [(None, 9948)]            0         
_________________________________________________________________
dense1 (Dense)               (None, 500)               4974500   
_________________________________________________________________
normalization1 (BatchNormali (None, 500)               2000      
_________________________________________________________________
dense2 (Dense)               (None, 500)               250500    
_________________________________________________________________
normalization2 (BatchNormali (None, 500)               2000      
_________________________________________________________________
dropout2 (Dropout)           (None, 500)               0         
_________________________________________________________________
embedding (Dense)            (None, 200)               100200 

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

earlystopper = EarlyStopping(
    monitor='val_loss', mode="min",
    patience = 5,
    verbose=1
    )

checkpointer = ModelCheckpoint(
    filepath = os.path.join(root_folder, "test_model"),
    monitor='val_loss', mode="min",
    verbose=1,
    save_best_only=True
    )

history = model.fit(train_data_generator,
          validation_data=val_data_generator,
          epochs=500, callbacks = [earlystopper, checkpointer])
model.save(os.path.join(root_folder, "test_ms2ds_model.hdf5"))

Epoch 1/500
  12/8795 [..............................] - ETA: 38:45 - loss: 0.0874

KeyboardInterrupt: 

In [10]:
print(history)

None


In [ ]:
from ms2deepscore.models import load_model

In [46]:
my_model = load_model(os.path.join(root_folder, "test_ms2ds_model.hdf5"))
print(my_model)

In [47]:
similarity_measure = MS2DeepScore(my_model)
score = similarity_measure.pair(training_spectra[0], training_spectra[1])
print(score)

Create BinnedSpectrum instances: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

0.8623855932103395


In [48]:
scores = similarity_measure.matrix(training_spectra[:3], training_spectra[:3])
print(scores)

Calculating vectors of reference spectrums: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 35.87it/s]

[[1.         0.8623856  0.47568729]
 [0.8623856  1.         0.63020755]
 [0.47568729 0.63020755 1.        ]]
